In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import difflib
import json

import context
from cont_gen.utils import load_json, load_jsonl

In [2]:
# Check new para_merge data
merge_data = load_jsonl('../../data/cuad_clean/CUADv1_paras_merge_new.jsonl')
print(len(merge_data))

In [6]:
# Check the offset
for cont in merge_data:
    for para in cont['paras']:
        p_text = para['text']
        for qa in para['qas']:
            for ans in qa['answers']:
                a_t = ans['text']
                p_span = p_text[ans['start_pos']: ans['end_pos'] + 1]
                assert a_t == p_span
